In [1]:
import numpy as np
import pandas as pd
import os
# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.svm import LinearSVR, SVR, SVC,LinearSVC
from sklearn.datasets import make_regression
from sklearn.preprocessing import normalize 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression, RidgeClassifier, RidgeClassifierCV
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from utils import dataset_imputer, get_score

In [2]:
# Read in data
df_training_features = pd.read_csv('train_features.csv')
df_training_labels = pd.read_csv('train_labels.csv')
all_pids = [pid for pid in df_training_features['pid'].unique()]

pids_train, pids_val = train_test_split(all_pids, test_size = 0.1)

In [3]:
# def getPatientData(trainingData, pids, patients=0, mode='pid'):
#     if mode == 'number':
#         pids = all_pids[:patients]
#     if len(pids) == 0:
#         return trainingData
#     #pids = np.array(pids).astype(np.float)
#     patients = [trainingData.iloc[idx] for idx in range(0, len(trainingData)) if trainingData['pid'][idx] in pids]    
#     #patientTrainingDataIndex = [trainingData.iloc[idx] for idx, col in enumerate(trainingData) if trainingData['pid'][idx] in pids]    
#     return pd.DataFrame(patients)

# def partitionData(trainingDataPids, trainingPartition=80):
#     validationPartition = 100 - trainingPartition
#     countTraining = int((trainingPartition/100)*len(trainingDataPids))
#     training = trainingDataPids[:countTraining]
#     validation = trainingDataPids[countTraining:]
#     print('')
#     print('Training size: ' + str(countTraining))
#     print('Validation size: ' + str(len(validation)))
#     return training, validation

# def populateData(X,Y):
#     Z = pd.merge(X, Y, on='pid')
#     YData = Z[Y.columns].iloc[:,1:]
#     XData = Z[X.columns].iloc[:,1:]
#     return XData, YData
# import sklearn.metrics as metrics

# TESTS = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total',
#          'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2',
#          'LABEL_Bilirubin_direct', 'LABEL_EtCO2']
# #def get_score(df_true, df_submission):
#     df_submission = df_submission.sort_values('pid')
#     df_true = df_true.sort_values('pid')
    
#     #task1 = np.mean([metrics.roc_auc_score(df_true[entry], df_submission[entry]) for entry in TESTS])
#     task2 = metrics.roc_auc_score(df_true['LABEL_Sepsis'], df_submission['LABEL_Sepsis'])
#     #task3 = np.mean([0.5 + 0.5 * np.maximum(0, metrics.r2_score(df_true[entry], df_submission[entry])) for entry in VITALS])
#     #score = np.mean([task1, task2, task3])
#     return task2

IndentationError: unexpected indent (<ipython-input-3-76159bbf12be>, line 32)

In [4]:
X_pid_train = dataset_imputer(df_training_features, method='mean_and_count', pid_list=pids_train, fillna=True)
X_pid_val = dataset_imputer(df_training_features, method='mean_and_count', pid_list=pids_val, fillna=True)


0.0 %
10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %
100.0 % - completed

0.0 %
10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %
100.0 % - completed



In [5]:
Y_pid_train = dataset_imputer(df_training_labels, method=None, pid_list=pids_train, fillna=True)
Y_pid_val = dataset_imputer(df_training_labels, method=None, pid_list=pids_val, fillna=True)

100.0 % - completed

100.0 % - completed



In [26]:
x_train, x_val = X_pid_train.fillna(0).copy(), X_pid_val.fillna(0).copy()
y_train, y_val = Y_pid_train.fillna(0).copy(), Y_pid_val.fillna(0).copy()

feature_columns = ['pid', 'PTT', 'HCO3', 'BaseExcess', 'PaCO2', 'FiO2', 'SaO2','Chloride', 'Hct', 'pH'
                      , 'EtCO2_count',
                       'PTT_count', 'BUN_count', 'Lactate_count', 'Temp_count', 'Hgb_count',
                       'HCO3_count', 'BaseExcess_count', 'RRate_count', 'Fibrinogen_count',
                       'Phosphate_count', 'WBC_count', 'Creatinine_count', 'PaCO2_count',
                       'AST_count', 'FiO2_count', 'Platelets_count', 'SaO2_count',
                       'Glucose_count', 'ABPm_count', 'Magnesium_count', 'Potassium_count',
                       'ABPd_count', 'Calcium_count', 'Alkalinephos_count', 'SpO2_count',
                       'Bilirubin_direct_count', 'Chloride_count', 'Hct_count',
                       'Heartrate_count', 'Bilirubin_total_count', 'TroponinI_count',
                       'ABPs_count', 'pH_count']

x_train, x_val = x_train[feature_columns], x_val[feature_columns]


label_columns = ['pid', 'LABEL_Sepsis']

y_train, y_val = y_train[label_columns], y_val[label_columns]

print(x_train.head())
print('*'*100)
print(y_train.head())

    pid   PTT  HCO3  BaseExcess  PaCO2      FiO2  SaO2  Chloride    Hct  \
0   2.0  31.8   0.0    0.000000    0.0  0.000000   0.0       0.0  40.20   
0   4.0  34.6   0.0    0.000000    0.0  0.000000   0.0       0.0  33.55   
0   6.0  53.8  19.5   -2.857143   39.0  0.566667  95.0     113.5  30.39   
0   8.0   0.0   0.0    0.000000    0.0  0.000000   0.0       0.0   0.00   
0  10.0  27.8   0.0    0.000000    0.0  0.000000   0.0       0.0  42.10   

         pH    ...     Alkalinephos_count  SpO2_count  Bilirubin_direct_count  \
0  0.000000    ...                    0.0        11.0                     0.0   
0  0.000000    ...                    1.0        11.0                     1.0   
0  7.352857    ...                    0.0        12.0                     0.0   
0  0.000000    ...                    0.0        10.0                     0.0   
0  0.000000    ...                    1.0        11.0                     0.0   

   Chloride_count  Hct_count  Heartrate_count  Bilirubin_total

In [27]:
print(x_train.shape)
print(y_train.shape)

(17095, 44)
(17095, 2)


In [30]:
# Train MLPClassifier
#regr =   KNeighborsClassifier(3)
regr = MLPClassifier(alpha=1e-4, hidden_layer_sizes=(200,200,200), random_state=1, solver='sgd', max_iter=300)
#regr = RidgeClassifierCV()
#regr = RandomForestClassifier()
regr.fit(x_train.iloc[:,1:], y_train.iloc[:,1].values)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(200, 200, 200), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=300,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='sgd',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [39]:
f = regr.predict_proba(x_val.iloc[:,1:])
f = pd.DataFrame(f)
f.columns = ['False', 'LABEL_Sepsis']
f.drop(['False'], axis=1, inplace=True)
f['pid'] = x_val.iloc[:,0].reset_index(drop=True)

print(get_score(y_val, f, tasks=['task2'])[1])

0.6930786576469028


In [2]:
# TEST TIME!

df_test_features = pd.read_csv('test_features.csv')

all_pids_test = [pid for pid in df_test_features['pid'].unique()]

X_pid_test = dataset_imputer(df_test_features, method='mean_and_count', pid_list=all_pids_test, fillna=True)

x_test = X_pid_test.fillna(0).copy()

feature_columns = ['pid', 'PTT', 'HCO3', 'BaseExcess', 'PaCO2', 'FiO2', 'SaO2','Chloride', 'Hct', 'pH'
                      , 'EtCO2_count',
                       'PTT_count', 'BUN_count', 'Lactate_count', 'Temp_count', 'Hgb_count',
                       'HCO3_count', 'BaseExcess_count', 'RRate_count', 'Fibrinogen_count',
                       'Phosphate_count', 'WBC_count', 'Creatinine_count', 'PaCO2_count',
                       'AST_count', 'FiO2_count', 'Platelets_count', 'SaO2_count',
                       'Glucose_count', 'ABPm_count', 'Magnesium_count', 'Potassium_count',
                       'ABPd_count', 'Calcium_count', 'Alkalinephos_count', 'SpO2_count',
                       'Bilirubin_direct_count', 'Chloride_count', 'Hct_count',
                       'Heartrate_count', 'Bilirubin_total_count', 'TroponinI_count',
                       'ABPs_count', 'pH_count']

x_test= x_test[feature_columns]

print(x_test.head())
print('*'*100)

f = regr.predict_proba(x_test.iloc[:,1:])
f = pd.DataFrame(f)
f.columns = ['False', 'LABEL_Sepsis']
f.drop(['False'], axis=1, inplace=True)
f['pid'] = x_test.iloc[:,0].reset_index(drop=True)

f.to_csv('task2_test.csv', index=None)

0.0 %
10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %
100.0 %
100.0 % - completed

   pid        PTT       HCO3  BaseExcess  PaCO2  FiO2  SaO2    Chloride  \
0  0.0  41.350000  14.500000        -6.5   28.0  0.53   0.0  100.000000   
0  3.0   0.000000   0.000000         0.0    0.0  0.00   0.0    0.000000   
0  5.0   0.000000   0.000000         0.0    0.0  0.00   0.0    0.000000   
0  7.0  75.766667  24.666667         0.5   43.0  0.50  70.0  100.666667   
0  9.0  35.400000  22.000000         0.0    0.0  0.00   0.0  103.000000   

     Hct    pH  ...  Alkalinephos_count  SpO2_count  Bilirubin_direct_count  \
0  28.45  7.37  ...                 2.0         9.0                     0.0   
0   0.00  0.00  ...                 0.0         6.0                     0.0   
0   0.00  0.00  ...                 0.0        11.0                     0.0   
0  22.99  7.38  ...                 3.0        11.0                     2.0   
0  33.70  0.00  ...                 0.0        11.0     

NameError: name 'regr' is not defined